<a href="https://colab.research.google.com/github/MAVRICK-1/ML-pracyice-/blob/master/api_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import requests as req

# Fetch files

In [10]:
data = req.get("https://www.omdbapi.com/?i=tt0944947&Season=1&apikey=bdd6ec2b")
data
if data.status_code == 200:
    print("Success")

Success


In [23]:
df = pd.DataFrame()
i=1

while True:
    temp_data = req.get(f"https://www.omdbapi.com/?i=tt0944947&Season={i}&apikey=bdd6ec2b")
    if temp_data.status_code == 200:
        temp_json = temp_data.json()

        # Check if "Episodes" key exists in the response
        if "Episodes" in temp_json:
            temp_df = pd.DataFrame(temp_json["Episodes"])
            i += 1

            # Concatenate the new DataFrame
            df = pd.concat([df, temp_df], ignore_index=True)
        else:
            # Break if "Episodes" key is missing (likely no more seasons)
            break
    else:
        # Break on HTTP error
        break



                                    Title    Released Episode imdbRating  \
0                  Unaired Original Pilot         N/A       0        N/A   
1                        Winter Is Coming  2011-04-17       1        8.9   
2                           The Kingsroad  2011-04-24       2        8.6   
3                               Lord Snow  2011-05-01       3        8.5   
4   Cripples, Bastards, and Broken Things  2011-05-08       4        8.6   
..                                    ...         ...     ...        ...   
69         A Knight of the Seven Kingdoms  2019-04-21       2        7.9   
70                         The Long Night  2019-04-28       3        7.5   
71                 The Last of the Starks  2019-05-05       4        5.5   
72                              The Bells  2019-05-12       5        5.9   
73                        The Iron Throne  2019-05-19       6        4.0   

        imdbID  
0   tt31321401  
1    tt1480055  
2    tt1668746  
3    tt1829962  
4 

The error in your code lies in how the response data from the OMDB API (`temp_data.json()`) is being processed into a DataFrame. The `temp_data.json()` response is likely a nested dictionary, and passing it directly to `pd.DataFrame` can lead to unexpected results or errors.

Here’s a detailed breakdown of the issues and the corrections:

### Issue 1: Incorrect handling of JSON data
The OMDB API response for a season request likely contains keys like `Season`, `Episodes`, etc., where `Episodes` is a list of dictionaries. When you try to create a DataFrame with `temp_data.json()`, it might not directly map to the expected tabular format.

### Solution:
You need to extract the relevant part of the JSON (`Episodes`) before creating the DataFrame.

### Corrected Code:
```python
import pandas as pd
import requests as req

df = pd.DataFrame()
i = 1

while True:
    temp_data = req.get(f"https://www.omdbapi.com/?i=tt0944947&Season={i}&apikey=bdd6ec2b")
    if temp_data.status_code == 200:
        temp_json = temp_data.json()
        
        # Check if "Episodes" key exists in the response
        if "Episodes" in temp_json:
            temp_df = pd.DataFrame(temp_json["Episodes"])
            i += 1
            
            # Concatenate the new DataFrame
            df = pd.concat([df, temp_df], ignore_index=True)
        else:
            # Break if "Episodes" key is missing (likely no more seasons)
            break
    else:
        # Break on HTTP error
        break

# Final DataFrame
print(df)
```

### Key Changes:
1. **Extract `Episodes`:** Used `temp_json["Episodes"]` to isolate the list of episodes, which can be converted into a DataFrame.
2. **Check for Missing Data:** Included a check for the presence of the `Episodes` key to handle cases where the season data might not be available.
3. **Avoid Infinite Loop:** Break out of the loop when no more seasons are available.

### Testing the Code:
Ensure the API key (`bdd6ec2b`) is valid and has sufficient quota. If the problem persists, check the API documentation to confirm the structure of the response for `Season` queries.

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Title       74 non-null     object
 1   Released    74 non-null     object
 2   Episode     74 non-null     object
 3   imdbRating  74 non-null     object
 4   imdbID      74 non-null     object
dtypes: object(5)
memory usage: 3.0+ KB


In [27]:
# prompt: out put to csv

df.to_csv('output.csv', index=False)